- Goal
    - Create synthetic data using IDM
    - Learn parameters of that data using particle filtering
    - Aliter 7 Feb: Learn using CEM idea, fitness function and then distb and then sample
- Learning
    - Need at least 2 vehicles so that there is a neighbor in the front
    - Otherwise src/1d/driver/lane_follower_driver errors
    - That is why the `AutomotiveDrivingModels/doc/1DMobius` stuff does 
    not work with 1 car only (needs at least 2 cars)
    - The tutorial does not work with `gen_straight_roadway` because that
    generates a roadway of type AutomotiveDrivingModels.roadway as opposed to
    AutomotiveDrivingModels.StraightRoadway
- Open question
    - IDM won't work unless there is a car in front (errors saynig nothing in sight)
    - We are focusing on learning the params of the second car here. Is that sound sensible?
- Flow of code
    - Call the required `usings`
    - Define functions required
    - Actual running things
        - Generate true trajectory
        - Generate a set of particles uniformly between sensible range of values
        - Compute fitness, sort and select numtop
        - Fit a distribution over this
        - Resample particles
- Scenario
    - Slower car in front (car 1)
    - Faster car behind (car 2)
    - We want to estimate params of car 2
- Feb 10: Adding timegap_des as our 2nd param in the 2 car, 1D, IDM scenario
    - Make code capable of handling 2d param. So far, had only scalar param
    i.e v_des
    - Fitting 2D distributions is required now
- Feb 22
    - the `rec` generated using `simulate` is a devious monster
    - It stores the last timestep in the 1st entry and first timestep in last entry
    - Be careful

In [1]:
using AutomotiveDrivingModels
using AutoViz
using Reel
using Interact
using StatsBase # For random particle generation
using Test

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/ac92513fc97126f4d4502d0637c43de8cdb5fca9-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/ac92513fc97126f4d4502d0637c43de8cdb5fca9-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [115]:
using Distributions

In [123]:
using Pkg

In [124]:
Pkg.dir("AutomotiveDrivingModels")
# Remember to git checkout the correct branch in ~/.julia/dev/AutomotiveDrivingModels.jl

┌ Warning: `Pkg.dir(pkgname, paths...)` is deprecated; instead, do `import AutomotiveDrivingModels; joinpath(dirname(pathof(AutomotiveDrivingModels)), "..", paths...)`.
└ @ Pkg.API /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.1/Pkg/src/API.jl:395


"/home/raunak/.julia/dev/AutomotiveDrivingModels/"

In [2]:
# 1 lane, 1000 m roadway
# roadway = gen_straight_roadway(1,1000.0) # Does not work with the below because AutomotiveDrivingModels.Roadway
roadway = StraightRoadway(1000.0); # AutomotiveDrivingModels.StraightRoadway type

In [3]:
"""
init_scene: Initialize a scene given positions and velocities of 2 cars.

BREAKING POSS:
- Hard coded for 2 vehicles
"""
# Function: Initialize the scene
    # Note: Vehicle def 1 (leader) is the one we learn params for
    # This is hard-coded in the gen_traj method by accessing first elem of scene vector
# The returned thing here is an array of entities i.e. cars and you can query those to get pos, vel
function init_scene(pos1=50.0,vel1=12.0,pos2=10.0,vel2=10.0)
    scene = Scene1D()
    
    # First arg to State1D is pos, 2nd arg is velocity
    push!(scene, Entity(State1D(pos1, vel1), VehicleDef(), 1))
    push!(scene, Entity(State1D(pos2, vel2), VehicleDef(), 2))
    return scene
end

init_scene (generic function with 5 methods)

In [8]:
"""
gen_traj: Generate trajectory using a given particle for default 100 timestep trajectory

-----------Arguments:
`particle` Dict with keys as parameter names of IDM and values as parameter values
`nticks` Duration of simulation

-----------Other functions called:`init_scene`
"""
function gen_traj(particle;nticks=100,timestep=0.1)
    scene = init_scene()
    models = Dict{Int, LaneFollowingDriver}()
    models[1] = IntelligentDriverModel(;particle...)
    models[2] = IntelligentDriverModel(v_des=12.0)

    # Simulate for nticks (default 100) time steps
    timestep = 0.1
    rec = QueueRecord(Vehicle1D, nticks+1, timestep)
    simulate!(LaneFollowingAccel, rec, scene, roadway, models, nticks)

    # Extract the position and velocity of nticks timesteps
    # X stores this as 2d array. Timestep is the row, col 1 is pos,vel is pos2
    n_cars = scene.n
    n_ticks = nticks
    X = Array{Float64}(undef, n_ticks, 1)

    for t in 1:n_ticks
        f = rec.frames[n_ticks - t + 1]
        
        # BAD: 2 cars in scene therefore loop has only 1 elem
        for c in 1:1 #Was 2:sc
            s = f.entities[c].state
            X[t, 1] = s.s #position
        end
    end
    return X, rec
end

gen_traj

In [10]:
"""
hallucinate_a_step: given positions and velocities of two cars, propagate scene one step forward
-----------Arguments:
`car1` Has type entity which can be obtained from queuerecord
`particle` Dict with key and IDM parameter name and value as param val

-----------Other functions called: `init_scene`
"""
function hallucinate_a_step(car1,car2,particle)
    scene = init_scene(car1.s,car1.v,car2.s,car2.v)
    models = Dict{Int, LaneFollowingDriver}()
    models[1] = IntelligentDriverModel(;particle...)
    models[2] = IntelligentDriverModel(v_des=12.0)

    # Simulate for nticks time steps
    nticks = 1
    timestep = 0.1
    rec = QueueRecord(Vehicle1D, nticks+1, timestep)
    simulate!(LaneFollowingAccel, rec, scene, roadway, models, nticks)

    # Extract the position and velocity of nticks timesteps
    # X stores this as 2d array. Timestep is the row, col 1 is pos,vel is pos2
    n_cars = scene.n
    n_ticks = nticks
    X = Array{Float64}(undef,n_ticks, 1)

    for t in 1:n_ticks
        f = rec.frames[n_ticks - t + 1]
        
        # BAD: 2 cars in scene therefore loop has only 1 elem
        for c in 1:1 #Was 2:sc
            s = f.entities[c].state
            X[t, 1] = s.s #position
        end
    end
    return X
end

hallucinate_a_step

In [118]:
"""
update_p_one_step: Update particles given one step of true data

------------Data structures that need explanation:
p_set_dict: Keys are param names. Corresponding value is array each element for different particle
vec_val_vec: Vector of value_vectors: Array with each element being array of values

------------Arguments:
`p_set_dict` Dictionary with parameters of IDM as keys and associated value as array of particles
`f` Frame to start hallucination from
`trupos` Resulting true position starting from frame f
`approach` Select "pf" or "cem"
`elite_fraction_percent` Required for the cem method to fit a distribution

------------Other functions called:`hallucinate_a_step`

------------Returns:
`new_p_set_dict` Dictionary with keys as IDM parameters and values as array of particles
"""
function update_p_one_step(p_set_dict,f,trupos;approach="pf",elite_fraction_percent=20)
    # Get the number of particles
    num_p = -100
    num_params = length(keys(p_set_dict))
    timestep = 0.1 #TODO: Hardcoding remove
    
    # Extract keys (i.e. params) and corresponding array of values
    params = Array{Symbol}(undef,num_params,1)
    vec_val_vec = Array{Array}(undef,num_params,1) #Array containing associated values for each key
    for (kk,kv) in enumerate(p_set_dict)
        num_p = length(kv[2])
        params[kk] = kv[1]
        vec_val_vec[kk] = kv[2]
    end
    
    #Loop over the particles and score each of them
    lkhd_vec = Array{Float64}(undef,num_p)
    for i in 1:num_p
        p_dict = Dict()
        for j in 1:num_params
            p_dict[params[j]]=vec_val_vec[j][i]
        end
        
        std_dev_acc = p_dict[:σ]
        
        # hack to avoid the std_dev_pos become negative and error Normal distb
        if std_dev_acc <= 0 std_dev_acc = 0.1 end

        std_dev_pos = timestep*timestep*std_dev_acc
            
        hpos = hallucinate_a_step(f.entities[1].state,f.entities[2].state,p_dict)
        
        lkhd_vec[i] = pdf(Normal(hpos[1],std_dev_pos),trupos[1])
    end

    # Create a matrix with different rows being different parameters and diff cols being diff particles
    p_mat = hcat(vec_val_vec...)'
    @show pmat
    #     @show vcat(p_mat,lkhd_vec') # Show likelihood along with particle matrix
    
    if approach=="pf"
        @show "Particle filtering approach"
        p_weight_vec = weights(lkhd_vec./sum(lkhd_vec)) # Convert to weights form to use julia sampling
        idx = sample(1:num_p,p_weight_vec,num_p)
        new_p_mat = p_mat[:,idx] #Careful that idx is (size,1) and not (size,2)
    end
    
    if approach=="cem"
        @show "CEM approach"
        sortedidx = sortperm(lkhd_vec,rev=true)
        numtop = convert(Int64,ceil(num_p*elite_fraction_percent/100.0))
        best_particles = p_mat[:,sortedidx[1:numtop]] # elite selection
#         @show best_particles
        p_distribution = fit(MvNormal,best_particles) # fit distb using elites
        new_p_mat = rand(p_distribution,num_p) # sample num_p new particles from dist
    end
    
    # Create a new dictionary with param and associated particle value array
#     @show p_mat
#     @show new_p_mat
    new_p_set_dict = Dict()
    for k in 1:num_params
        new_p_set_dict[params[k]] = new_p_mat[k,:]
    end
#     @show new_p_set_dict
    return new_p_set_dict
    
end

update_p_one_step

In [121]:
"""
Generate a ground truth trajectory of 100 timesteps.
Make your particle filter go step by step over this ground truth.
Hoepfully by the end of the trajectory, it has figured out the true parameters
"""
num_p = 100
# start:step:end and number of particles are the inputs to sample
v_particles = sample(10.0:1.0:30.0,num_p)
sig_particles = sample(0.1:0.1:1.0,num_p)
old_p_set_dict = Dict(:v_des=>v_particles,:σ=>sig_particles)

# Generate ground truth trajectory
pos_ground_truth,rec_ground_truth = gen_traj(Dict(:v_des=>19.0,:σ=>0.2))

# loop over the trajectory step by step
f_end_num = length(rec_ground_truth.frames)
    
for t in 1:f_end_num-1
#     @show t
    f = rec_ground_truth.frames[f_end_num - t + 1]
    trupos = pos_ground_truth[t]
    
    # Select approach either "cem" or "pf"
    new_p_set_dict = update_p_one_step(old_p_set_dict,f,trupos,approach="cem")
    old_p_set_dict = new_p_set_dict
end
    
#@show fit(MvNormal,old_p_mat) # Don't work because all elements identical
@show old_p_set_dict

pmat = [11.0 13.0 18.0 30.0 21.0; 0.8 0.7 0.8 0.7 0.4]
"CEM approach" = "CEM approach"
pmat = [11.0 13.0 18.0 30.0 21.0; 0.8 0.7 0.8 0.7 0.4]
"CEM approach" = "CEM approach"
pmat = [11.0 13.0 18.0 30.0 21.0; 0.8 0.7 0.8 0.7 0.4]
"CEM approach" = "CEM approach"
pmat = [11.0 13.0 18.0 30.0 21.0; 0.8 0.7 0.8 0.7 0.4]
"CEM approach" = "CEM approach"
pmat = [11.0 13.0 18.0 30.0 21.0; 0.8 0.7 0.8 0.7 0.4]
"CEM approach" = "CEM approach"
pmat = [11.0 13.0 18.0 30.0 21.0; 0.8 0.7 0.8 0.7 0.4]
"CEM approach" = "CEM approach"
pmat = [11.0 13.0 18.0 30.0 21.0; 0.8 0.7 0.8 0.7 0.4]
"CEM approach" = "CEM approach"
pmat = [11.0 13.0 18.0 30.0 21.0; 0.8 0.7 0.8 0.7 0.4]
"CEM approach" = "CEM approach"
pmat = [11.0 13.0 18.0 30.0 21.0; 0.8 0.7 0.8 0.7 0.4]
"CEM approach" = "CEM approach"
pmat = [11.0 13.0 18.0 30.0 21.0; 0.8 0.7 0.8 0.7 0.4]
"CEM approach" = "CEM approach"
pmat = [11.0 13.0 18.0 30.0 21.0; 0.8 0.7 0.8 0.7 0.4]
"CEM approach" = "CEM approach"
pmat = [11.0 13.0 18.0 30.0 21.0; 0.8 0.7 0

old_p_set_dict = Dict{Any,Any}(:v_des=>[19.0114, 19.0116, 19.0114, 19.0116, 19.0114, 19.0114, 19.0115, 19.0116, 19.0116, 19.0115, 19.0117, 19.0115, 19.0117, 19.0116, 19.0117, 19.0116, 19.0117, 19.0114, 19.0115, 19.0115, 19.0115, 19.0117, 19.0116, 19.0117, 19.0117, 19.0114, 19.0117, 19.0115, 19.0116, 19.0116, 19.0116, 19.0116, 19.0115, 19.0115, 19.0116, 19.0117, 19.0115, 19.0115, 19.0116, 19.0118, 19.0115, 19.0115, 19.0115, 19.0115, 19.0116, 19.0115, 19.0114, 19.0115, 19.0117, 19.0116, 19.0116, 19.0116, 19.0117, 19.0115, 19.0115, 19.0117, 19.0115, 19.0116, 19.0116, 19.0116, 19.0114, 19.0113, 19.0117, 19.0115, 19.0115, 19.0115, 19.0116, 19.0117, 19.0115, 19.0113, 19.0116, 19.0116, 19.0115, 19.0117, 19.0113, 19.0114, 19.0116, 19.0115, 19.0117, 19.0115, 19.0114, 19.0115, 19.0115, 19.0116, 19.0116, 19.0115, 19.0117, 19.0114, 19.0115, 19.0115, 19.0115, 19.0116, 19.0117, 19.0117, 19.0114, 19.0114, 19.0114, 19.0116, 19.0114, 19.0116],:σ=>[0.18035, 0.180348, 0.180349, 0.180348, 0.180349, 0.1803

Dict{Any,Any} with 2 entries:
  :v_des => [19.0114, 19.0116, 19.0114, 19.0116, 19.0114, 19.0114, 19.0115, 19.…
  :σ     => [0.18035, 0.180348, 0.180349, 0.180348, 0.180349, 0.18035, 0.180349…

# Visualize

In [ ]:
# Function: Return rec corresponding to generated traj
    # Will help visualizatoin
    # Calls init_scene
# Might be useful later
    # models[2] = IntelligentDriverModel(v_des=particle[1],s_min=particle[2],T=particle[3])
function gen_rec4vid(particle;nticks=100,timestep=0.1)
    scene = init_scene()
    models = Dict{Int, LaneFollowingDriver}()
    models[1] = IntelligentDriverModel(v_des=particle[1],σ = particle[2])
    models[2] = IntelligentDriverModel(v_des=12.0)

    # Simulate for nticks (default 100) time steps
    timestep = 0.1
    rec = QueueRecord(Vehicle1D, nticks+1, timestep)
    simulate!(LaneFollowingAccel, rec, scene, roadway, models, nticks)

    return rec
end

In [ ]:
# overlays = [TextOverlay(text=["$(veh.id)"], incameraframe=true,
#         pos=VecE2(veh.state.s-0.7, 3)) for veh in scene];
# render(scene, roadway, overlays, cam=cam, canvas_height=100)

cam = StaticCamera(VecE2(100.0,0.0), 4.75)
true_rec = gen_rec4vid([20.0 0.1],nticks=100)
rec = true_rec
@manipulate for frame_index in 1 : nframes(rec)
    render(rec[frame_index-nframes(rec)], roadway, cam=cam, canvas_height=100)
end

# LEARNING AND EXPERIMENTATION

In [ ]:
# LEARNING ABOUT MULTIDIM DISTB
# Test: Generate samples for a 2d distb
d2 = MvNormal(2,2.0) # first arg shows dimension, second shows std dev
qw = rand(d2,6) # Will generate 6 samples i.e. 6 columns

# Test: Fit 2d distribution
dx = Normal()
dy = Normal(2,1.0)
x = rand(dx,100)
y = rand(dy,100)

# Matrix with each column being a sample
# Total columns is total number of samples
# Total rows is number of parameters
# All entries in a row contain value from same param eg:v_des
data_matrix = vcat(x',y')
fit(MvNormal,data_matrix)

In [ ]:
using PyPlot

In [ ]:
num_samples = 2000
y1 = rand(Normal(10.0,5.0),num_samples)
y2 = rand(Normal(2.0,1.0),num_samples)
plot(1:num_samples,y1)
plot(1:num_samples,y2)

In [ ]:
using StatPlots

In [ ]:
StatPlots.plot(Normal(3,5),linewidth=4,size=(2500,2500))

In [ ]:
roadway = gen_straight_roadway(2,1000.0);

In [ ]:
scene = Scene1D()
push!(scene, Entity(State1D(10.0,  8.0), VehicleDef(), 1))
push!(scene, Entity(State1D(50.0, 12.5), VehicleDef(), 2))

cam = StaticCamera(VecE2(100.0,0.0), 4.75)
overlays = [TextOverlay(text=["$(veh.id)"], incameraframe=true, pos=VecE2(veh.state.s-0.7, 3)) for veh in scene]
render(scene, roadway, overlays, cam=cam, canvas_height=100)

# Tests (serves as usage examples for functions)

In [157]:
# init_scene
scene = init_scene()
@test isapprox(scene[1].state.s,50.0)

Test Passed

In [138]:
# gen_traj
particle = Dict(:v_des=>25.0,:σ=>0.5)
next_pos = gen_traj(particle,nticks=1)
@show next_pos
@test isapprox(next_pos[1][1],51.2)

next_pos = ([51.2168], QueueRecord(nframes=2))
Test Failed at In[138]:5
  Expression: isapprox((next_pos[1])[1], 51.2)
   Evaluated: isapprox(51.216824735640564, 51.2)


Test.FallbackTestSetException: There was an error during testing

In [131]:
# hallucinate_a_step
ptest = Dict(:v_des=>25.0,:σ=>0.5)
pos_ground_truth,rec_ground_truth = gen_traj(ptest)
frame_of_interest = rec_ground_truth.frames[101]
hallucination = hallucinate_a_step(frame_of_interest.entities[1].state,frame_of_interest.entities[2].state,ptest)
@test isapprox(hallucination[1],51.2)

Test Failed at In[131]:6
  Expression: isapprox(hallucination[1], 51.2)
   Evaluated: isapprox(51.214721633628656, 51.2)


Test.FallbackTestSetException: There was an error during testing

In [ ]:
# update_p_one_step
num_p = 5
# start:step:end and number of particles are the inputs to sample
v_particles = sample(10.0:1.0:30.0,num_p)
sig_particles = sample(0.1:0.1:1.0,num_p)
p_set_dict = Dict(:v_des=>v_particles,:σ=>sig_particles)
@show p_set_dict
# Generate ground truth trajectory
pos_ground_truth,rec_ground_truth = gen_traj(Dict(:v_des=>25.0,:σ=>0.2))
@show rec_ground_truth.frames[101].entities[1].state,pos_ground_truth[1]

update_p_one_step(p_set_dict,rec_ground_truth.frames[101],pos_ground_truth[1],
    approach="cem",elite_fraction_percent=60)

```
Expected output
p_set_dict = Dict(:v_des=>[11.0, 29.0, 27.0, 25.0, 30.0],:σ=>[1.0, 0.7, 0.4, 0.2, 0.7])
(rec_ground_truth.frames[101].entities[1].state, pos_ground_truth[1]) = 
(AutomotiveDrivingModels.State1D(50.0, 12.0), 51.2124675051445)
best_particles = [25.0 27.0 30.0; 0.2 0.4 0.7]
p_mat = [11.0 29.0 27.0 25.0 30.0; 1.0 0.7 0.4 0.2 0.7]
new_p_mat = [25.0191 27.8877 26.6066 26.3389 26.8579; 0.201913 0.488768 0.360664 0.333894 0.385794]
new_p_set_dict = Dict{Any,Any}(Pair{Any,Any}(:v_des, [25.0191, 27.8877, 26.6066, 26.3389, 26.8579]),
Pair{Any,Any}(:σ, [0.201913, 0.488768, 0.360664, 0.333894, 0.385794]))
```